# Quantum State Tomography

In [1]:
import qiskit
from qiskit_experiments.framework import ParallelExperiment
from qiskit_experiments.library import StateTomography

# For simulation
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeParis

# Noisy simulator backend
backend = AerSimulator.from_backend(FakeParis())

## State Tomography Experiment

To run a state tomography experiment we initialize the experiment with a circuit to prepare the state to be measured. We can also pass in an `Operator`, or a `Statevector` to describe the preparation circuit.

In [2]:
# Run experiments

# GHZ State preparation circuit
nq = 2
qc_ghz = qiskit.QuantumCircuit(nq)
qc_ghz.h(0)
qc_ghz.s(0)
for i in range(1, nq):
    qc_ghz.cx(0, i)

# QST Experiment
qstexp1 = StateTomography(qc_ghz)
qstdata1 = qstexp1.run(backend, seed_simulation=100).block_for_results()

# Print results
for result in qstdata1.analysis_results():
    print(result)

DbAnalysisResultV1
- name: state
- value: DensityMatrix([[ 0.48079427+0.j        , -0.00732422+0.00585937j,
                -0.00358073+0.01155599j,  0.00488281-0.44238281j],
               [-0.00732422-0.00585937j,  0.02604167+0.j        ,
                 0.00390625+0.00195312j,  0.00813802-0.01188151j],
               [-0.00358073-0.01155599j,  0.00390625-0.00195312j,
                 0.0218099 +0.j        ,  0.01416016-0.01269531j],
               [ 0.00488281+0.44238281j,  0.00813802+0.01188151j,
                 0.01416016+0.01269531j,  0.47135417+0.j        ]],
              dims=(2, 2))
- extra: <4 items>
- device_components: ['Q0', 'Q1']
- verified: False
DbAnalysisResultV1
- name: state_fidelity
- value: 0.9184570312499998
- device_components: ['Q0', 'Q1']
- verified: False
DbAnalysisResultV1
- name: positive
- value: True
- device_components: ['Q0', 'Q1']
- verified: False


### Tomography Results

The main result for tomography is the fitted state, which is stored as a `DensityMatrix` object:

In [3]:
state_result = qstdata1.analysis_results("state")
print(state_result.value)

DensityMatrix([[ 0.48079427+0.j        , -0.00732422+0.00585937j,
                -0.00358073+0.01155599j,  0.00488281-0.44238281j],
               [-0.00732422-0.00585937j,  0.02604167+0.j        ,
                 0.00390625+0.00195312j,  0.00813802-0.01188151j],
               [-0.00358073-0.01155599j,  0.00390625-0.00195312j,
                 0.0218099 +0.j        ,  0.01416016-0.01269531j],
               [ 0.00488281+0.44238281j,  0.00813802+0.01188151j,
                 0.01416016+0.01269531j,  0.47135417+0.j        ]],
              dims=(2, 2))


The state fidelity of the fitted state with the ideal state prepared by the input circuit is stored in the `"state_fidelity"` result field. Note that if the input circuit contained any measurements the ideal state cannot be automatically generated and this field will be set to `None`.

In [4]:
fid_result = qstdata1.analysis_results("state_fidelity")
print("State Fidelity = {:.5f}".format(fid_result.value))

State Fidelity = 0.91846


#### Additional state metadata

Additional data is stored in the tomography under the `"state_metadata"` field. This includes
- `eigvals`: the eigenvalues of the fitted state
- `trace`: the trace of the fitted state
- `positive`: Whether the eigenvalues are all non-negative
- `positive_delta`: the deviation from positivity given by 1-norm of negative eigenvalues.

If trace rescaling was performed this dictionary will also contain a `raw_trace` field containing the trace before rescaling.
Futhermore, if the state was rescaled to be positive or trace 1 an additional field `raw_eigvals` will contain the state eigenvalues before rescaling was performed.

In [5]:
state_result.extra

{'fitter': 'linear_inversion',
 'fitter_time': 0.002582073211669922,
 'eigvals': array([0.91857346, 0.05769817, 0.02190643, 0.00182194]),
 'trace': 0.9999999999999998}

To see the effect of rescaling we can perform a "bad" fit with very low counts

In [6]:
# QST Experiment
bad_data = qstexp1.run(backend, shots=10, seed_simulation=100).block_for_results()
bad_state_result = bad_data.analysis_results("state")

# Print result
print(bad_state_result)

# Show extra data
bad_state_result.extra

DbAnalysisResultV1
- name: state
- value: DensityMatrix([[ 0.48229097+0.j        , -0.05136656-0.01922528j,
                -0.00576701+0.05963492j, -0.2045793 -0.32837222j],
               [-0.05136656+0.01922528j,  0.12109684+0.j        ,
                -0.04873269-0.00115347j, -0.04065805-0.00651795j],
               [-0.00576701-0.05963492j, -0.04873269+0.00115347j,
                 0.02690666+0.j        , -0.00884281+0.04642445j],
               [-0.2045793 +0.32837222j, -0.04065805+0.00651795j,
                -0.00884281-0.04642445j,  0.36970552+0.j        ]],
              dims=(2, 2))
- extra: <5 items>
- device_components: ['Q0', 'Q1']
- verified: False


{'fitter': 'linear_inversion',
 'fitter_time': 0.0013988018035888672,
 'eigvals': array([0.82391588, 0.17608412, 0.        , 0.        ]),
 'raw_eigvals': array([ 0.88353364,  0.23570189,  0.03766169, -0.15689722]),
 'trace': 0.999999999999999}

## Tomography Fitters

The default fitters is `linear_inversion`, which reconstructs the state using *dual basis* of the tomography basis. This will typically result in a non-postive reconstructed state. This state is rescaled to be postive-semidfinite (PSD) by computing its eigen-decomposition and rescaling its eigenvalues using the approach from *J Smolin, JM Gambetta, G Smith, Phys. Rev. Lett. 108, 070502 (2012), [open access](https://arxiv.org/abs/arXiv:1106.5458).

There are several other fitters are included (See API documentation for details). For example if `cvxpy` is installed we can use the `cvxpy_gaussian_lstsq` fitter which allows constraining the fit to be PSD without requiring rescaling.

In [7]:
try:
    import cvxpy
    
    # Set analysis option for cvxpy fitter
    qstexp1.set_analysis_options(fitter='cvxpy_gaussian_lstsq')
    
    # Re-run experiment
    qstdata2 = qstexp1.run(backend, seed_simulation=100).block_for_results()

    state_result2 = qstdata2.analysis_results("state")
    print(state_result2)   
    print("\nextra:")
    for key, val in state_result2.extra.items():
        print(f"- {key}: {val}")

except ModuleNotFoundError:
    print("CVXPY is not installed")

DbAnalysisResultV1
- name: state
- value: DensityMatrix([[ 0.49424158+0.00000000e+00j, -0.00841736+1.99277568e-04j,
                 0.00598117+5.72570156e-03j,  0.00185319-4.50350231e-01j],
               [-0.00841736-1.99277568e-04j,  0.02760711+0.00000000e+00j,
                 0.00588045-6.01534114e-03j,  0.01346635-1.33325538e-02j],
               [ 0.00598117-5.72570156e-03j,  0.00588045+6.01534114e-03j,
                 0.01842177+0.00000000e+00j,  0.01351345-2.41429664e-02j],
               [ 0.00185319+4.50350231e-01j,  0.01346635+1.33325538e-02j,
                 0.01351345+2.41429664e-02j,  0.45972953+0.00000000e+00j]],
              dims=(2, 2))
- extra: <8 items>
- device_components: ['Q0', 'Q1']
- verified: False

extra:
- cvxpy_solver: CVXOPT
- cvxpy_status: optimal
- fitter: cvxpy_gaussian_lstsq
- fitter_time: 0.03557395935058594
- eigvals: [0.9282924  0.05504855 0.01665905 0.        ]
- raw_eigvals: [ 9.27753637e-01  5.50166071e-02  1.66493815e-02 -1.24969847e-08]
- tr

## Parallel Tomography Experiment

We can also use the `qiskit_experiments.ParallelExperiment` class to run subsystem tomography on multiple qubits in parallel.

For example if we want to perform 1-qubit QST on several qubits at once:

In [8]:
from math import pi
num_qubits = 5
gates = [qiskit.circuit.library.RXGate(i * pi / (num_qubits - 1))
         for i in range(num_qubits)]

subexps = [
    StateTomography(gate, qubits=[i])
    for i, gate in enumerate(gates)
]
parexp = ParallelExperiment(subexps)
pardata = parexp.run(backend, seed_simulation=100).block_for_results()

for result in pardata.analysis_results():
    print(result)

DbAnalysisResultV1
- name: parallel_experiment
- value: 5
- extra: <2 items>
- device_components: ['Q0', 'Q1', 'Q2', 'Q3', 'Q4']
- verified: False


View component experiment analysis results

In [9]:
for i in range(parexp.num_experiments):
    expdata = pardata.component_experiment_data(i)
    state_result_i = expdata.analysis_results("state")
    fid_result_i = expdata.analysis_results("state_fidelity")
    
    print(f'\nPARALLEL EXP {i}')
    print("State Fidelity: {:.5f}".format(fid_result_i.value))
    print("State: {}".format(state_result_i.value))


PARALLEL EXP 0
State Fidelity: 0.98633
State: DensityMatrix([[0.98632812+0.j        , 0.0078125 -0.00195312j],
               [0.0078125 +0.00195312j, 0.01367188+0.j        ]],
              dims=(2,))

PARALLEL EXP 1
State Fidelity: 0.98545
State: DensityMatrix([[0.84765625+0.j        , 0.00585938+0.33886719j],
               [0.00585938-0.33886719j, 0.15234375+0.j        ]],
              dims=(2,))

PARALLEL EXP 2
State Fidelity: 0.97070
State: DensityMatrix([[0.52050781+0.j        , 0.01074219+0.47070313j],
               [0.01074219-0.47070313j, 0.47949219+0.j        ]],
              dims=(2,))

PARALLEL EXP 3
State Fidelity: 0.98337
State: DensityMatrix([[ 0.16601563+0.j        , -0.01953125+0.34960938j],
               [-0.01953125-0.34960938j,  0.83398437+0.j        ]],
              dims=(2,))

PARALLEL EXP 4
State Fidelity: 0.97559
State: DensityMatrix([[ 0.02441406+0.j        , -0.01855469-0.01074219j],
               [-0.01855469+0.01074219j,  0.97558594+0.j        ]],
  